In [6]:
import sys
from os.path import dirname, abspath
#_PARENT_DIR = dirname(dirname(dirname(abspath(__file__))))
#sys.path.append(_PARENT_DIR)
#from utils._utils import parse_celonis_url, get_user_data, get_logger              ### <--
from pycelonis import get_celonis
import argparse
#logger = get_logger()
def unify_tables(url, api_token, datapool, tables, tableSchema, function = 'null'):
    
    #loop through defined tables
    #for table_name in tables:
       
    c = get_celonis(url, api_token)
    #get_user_data(c, 'Table Unifier')
    #url_options= parse_celonis_url(url)                                            ### <--
    #pool = c.pools.find(url_options['id'])                                         ### <--
    
    pool = c.pools.find(datapool)                                                   ### <--

    minReq = []
    maxReq = []
    sameTables = []
    schemalist = tableSchema                                                        # added to extract required table schemas (in python list format)
    columns = ''
    select = ''
    
    # create or find global data job 
    try:
        global_dj = pool.data_jobs.find('Unify Tables')
    except:
        global_dj = pool.create_data_job(name = 'Unify Tables')
    
    #loop through defined tables
    for table_name in tables:
    
        # find tables to unify
        x = 0
        for schema in global_dj.tables:
            for table in schema:
                x = x + 1
                if table['schemaName'] not in schemalist:
                    continue
                if table['name'] == table_name:
                    sameTables.append(table)
        
        if len(sameTables) == 0:
            print('TABLE', table_name, 'CAN NOT BE UNIFIED! Not existing in source systems!')
            global_dj.create_transformation(name = "Unify " + table_name, statement = '-- no existing tables to unify in defined source systems')
            continue

        else:
            # create minimal column requirement
            for col in sameTables[0]['columns']:
                try:
                    if col in sameTables[1]['columns']:                                              # ERROR CHECKING !
                        minReq.append(col)
                except:
                    continue

            """
            #only required if function = 'min'

            if len(sameTables) > 2:
                for table in sameTables[2:]:
                    for col in minReq:
                        if col not in table['columns']:
                            minReq.pop(col)
                # create transformation statement for minimal table requirement

            if function == 'min':
                for col in minReq:
                    if(minReq.index(col) == 0):
                        columns += '{}\n'.format(col['name'])
                    else:
                        columns += ',{}\n'.format(col['name'])
                for table in sameTables:
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            """            

            # create transformation for statement for extension with null
            if function == 'null':
                #columns = ''
                #select = ''
                
                for table in sameTables:
                    for col in table['columns']:
                        if col not in maxReq:
                            maxReq.append(col)
                for table in sameTables:
                    for col in maxReq:
                        if(maxReq.index(col) == 0):
                            columns += "'" + table['schemaName'] + "' " + 'AS "SCHEMA"\n' + ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                        else:
                            columns += ',{}\n'.format('"' + col['name'] + '"' if col in table['columns'] else 'NULL AS "' + col['name'] + '"')
                    select += 'SELECT \n{}FROM "{}"."{}"'.format(columns, table['schemaName'], table_name)
                    columns = ''
                    if table != sameTables[len(sameTables)-1]:
                        select += '\nUNION ALL\n'
            # create transformation
            #statement = 'DROP VIEW IF EXISTS "{}_UNIFIED";\n\nCreate VIEW "{}_UNIFIED" AS(\n{}\n);'.format(table_name, table_name, select)
            statement = 'DROP VIEW IF EXISTS "{}";\n\nCreate VIEW "{}" AS(\n{}\n);'.format(table_name, table_name, select)                      ### <-- Unified
            global_dj.create_transformation(name = "Unify " + table_name, statement = statement)
            print('The union statement has been saved in the global data job "Unify Tables" for Table', table_name)
            select = ''
            sameTables = []
            minReq = []
            maxReq = []
            
            # reminder: datatypes

In [5]:
url = 'https://endresshauser.eu-3.celonis.cloud/'

# Specify a valid API token for your Cloud Team.
#Click on Team Settings -> Edit -> create or copy already inserted API key

api_token = 'NDEzZDFkN2YtZjVmZC00MTE0LTg2ODAtNmZlZDM5YzU5YTEwOjBxR0xOSFhjdDQ4MFNDc0V1UDIwbldzNWd2VXFVeUZieCt5MjdzbUhySkRP'

In [6]:
celonis = get_celonis()

2020-09-10 14:27:55 - pycelonis: Login successful! The Application Key currently has access to 0 Analyses and to 4 Data Pools.


In [7]:
  c = get_celonis(url, api_token)

2020-09-10 14:27:58 - pycelonis: Login successful! The Application Key currently has access to 0 Analyses and to 4 Data Pools.


In [9]:
celonis.pools

[<Pool, id e1bd9ca9-9d3b-44b4-9c17-ae345f663e27, name SAP ECC - Purchase to Pay>,
<Pool, id 590652c7-87a4-46e6-a9e9-60c90cb7c4e3, name SAP ECC - Production Planning>,
<Pool, id 98d9fc62-ccbf-4158-b89f-cb8a7a976a2d, name SAP ECC - Warehouse Management>,
<Pool, id 60442433-3dbb-4ce6-8ed3-e9bb7238c614, name SAP ECC - Order to Cash>,]

In [10]:
datapool = 'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27'

pool = celonis.pools.find(datapool)

In [11]:
pool.data_connections

[<DataConnection, id 77f18ab5-b465-4e03-8f67-0272cd01ddfb, name SAP ECC - PIN-020>,
<DataConnection, id 310d0ab5-5bb5-4d64-8a34-84bc93121d58, name SAP ECC - PIN-024>,
<DataConnection, id 8adecd8c-3535-43d2-bb7c-af12f175e8f5, name SAP ECC - PS1-033>,
<DataConnection, id a61e719d-5566-4328-b183-1d393f8879aa, name SAP ECC - PS1-035>,
<DataConnection, id b6fbc486-9719-4d69-87dc-50a471c5f95f, name SAP ECC - PS1-036>,
<DataConnection, id def198a6-426d-4ccb-b0ec-52dc2d1c519a, name SAP ECC - PS1-038>,
<DataConnection, id 98f12823-2c82-4df3-a4e0-3f932fb27879, name SAP ECC - PS1-039>,
<DataConnection, id d04e6dc0-5b53-420a-b624-8eed5d1b7db7, name SAP ECC - PS1-030>,
<DataConnection, id 621b2b5e-1c46-4e81-b13b-15bc607ff983, name SAP ECC - PS1-007>,
<DataConnection, id ffdff050-1fe4-43aa-ae9a-cd4763450e59, name SAP ECC - PS1-040>,
<DataConnection, id 5a2082de-1aec-4bff-850c-7a923f2d094b, name SAP ECC - PS1-044>,
<DataConnection, id 70f5784d-f0b6-4ce0-a368-6104c27a245b, name SAP ECC - PS1-032>,
<Da

In [7]:
# URL to the Cloud Data Pool where the tables to unify are hosted.
url = 'https://endresshauser.eu-3.celonis.cloud/'

# Specify a valid API token for your Cloud Team.
#Click on Team Settings -> Edit -> create or copy already inserted API key

api_token = 'NDEzZDFkN2YtZjVmZC00MTE0LTg2ODAtNmZlZDM5YzU5YTEwOjBxR0xOSFhjdDQ4MFNDc0V1UDIwbldzNWd2VXFVeUZieCt5MjdzbUhySkRP'


#To get the data pool id, you have to run the following code. To get the data
#the ML User here called Global needs permission to request these informations
#celonis = get_celonis()
#celonis.pools by that you get the ID for the data pool 

# Specify a datapool -- P2P Data Pool ID
datapool = 'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27'

# Specify the table names for which the tables should be merged. 

tables = ['BKPF','BSEG','CDHDR','CDPOS','DD02T','DD03M','DD07T','EBAN','EINA','EKBE','EKES','EKET','EKKO','EKPO','LFA1','LFB1','LFM1','LIKP','LTAK','LTAP','MAKT','MARA','MARC','MKPF','MSEG','NAST','QALS','QAVE','QPAM','QPCD','QPGR','RBKP','RSEG','T001','T001L','T001W','T008T','T023T','T024','T024E','T161T','T163I','T163Y','T16FE','T179T','T685T','TCURF','TCURR','TCURV','TCURX','USR02','VBFA','ZZFAX01', 'T320','T134T','T307T','T077Y']

# Specify 'min' if the unified table should be the minimal column requirement and 'null' if the not existing columns should be filled with null values.
function = 'null'

#Schema Information

#<DataConnection, id 77f18ab5-b465-4e03-8f67-0272cd01ddfb, name SAP ECC - PIN-020>,
#<DataConnection, id 310d0ab5-5bb5-4d64-8a34-84bc93121d58, name SAP ECC - PIN-024>,
#<DataConnection, id 8adecd8c-3535-43d2-bb7c-af12f175e8f5, name SAP ECC - PS1-033>,
#<DataConnection, id a61e719d-5566-4328-b183-1d393f8879aa, name SAP ECC - PS1-035>,
#<DataConnection, id b6fbc486-9719-4d69-87dc-50a471c5f95f, name SAP ECC - PS1-036>,
#<DataConnection, id def198a6-426d-4ccb-b0ec-52dc2d1c519a, name SAP ECC - PS1-038>,
#<DataConnection, id 98f12823-2c82-4df3-a4e0-3f932fb27879, name SAP ECC - PS1-039>,
#<DataConnection, id d04e6dc0-5b53-420a-b624-8eed5d1b7db7, name SAP ECC - PS1-030>,
#<DataConnection, id 621b2b5e-1c46-4e81-b13b-15bc607ff983, name SAP ECC - PS1-007>,
#<DataConnection, id ffdff050-1fe4-43aa-ae9a-cd4763450e59, name SAP ECC - PS1-040>,
#<DataConnection, id 5a2082de-1aec-4bff-850c-7a923f2d094b, name SAP ECC - PS1-044>,
#<DataConnection, id 70f5784d-f0b6-4ce0-a368-6104c27a245b, name SAP ECC - PS1-032>,
#<DataConnection, id 7982c009-f641-4fa7-b74c-4c79febc733a, name SAP ECC - PSG-045>,
#<DataConnection, id 00409fbb-dad1-444e-90c1-c026cf6b9588, name SAP ECC - PSG-048>,
#<DataConnection, id 90db209c-f801-4c03-90d2-dac4c191ba9b, name SAP ECC - PSG-049>,
#<DataConnection, id 7a3bafc7-ae0a-4c96-82bf-0e16ba8ff97d, name SAP ECC - PCA-042>,
#<DataConnection, id 9363d8e4-bbd1-4454-ab9f-b115ab31aa5c, name SAP ECC - PP1-011>,
#<DataConnection, id 1c0a359a-f05b-4fa7-8cac-b0018a68ee74, name SAP ECC - PP2-004>,
#<DataConnection, id 920b7d87-7487-47b2-b46a-0727145161ce, name SAP ECC - PP2-005>,
#<DataConnection, id b9ddf35b-8fa1-4f45-a41d-c873e600ca7c, name SAP ECC - PS1-090>



# Specify the required table schemas in a python list
#To acess the Schema Name you have to get the connection informations
#pool.data_connections
tableSchema = [
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_9363d8e4-bbd1-4454-ab9f-b115ab31aa5c',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_77f18ab5-b465-4e03-8f67-0272cd01ddfb',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_310d0ab5-5bb5-4d64-8a34-84bc93121d58',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_8adecd8c-3535-43d2-bb7c-af12f175e8f5',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_a61e719d-5566-4328-b183-1d393f8879aa',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_b6fbc486-9719-4d69-87dc-50a471c5f95f',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_def198a6-426d-4ccb-b0ec-52dc2d1c519a',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_98f12823-2c82-4df3-a4e0-3f932fb27879',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_d04e6dc0-5b53-420a-b624-8eed5d1b7db7',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_621b2b5e-1c46-4e81-b13b-15bc607ff983',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_ffdff050-1fe4-43aa-ae9a-cd4763450e59',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_5a2082de-1aec-4bff-850c-7a923f2d094b',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_70f5784d-f0b6-4ce0-a368-6104c27a245b',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_7982c009-f641-4fa7-b74c-4c79febc733a',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_00409fbb-dad1-444e-90c1-c026cf6b9588',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_90db209c-f801-4c03-90d2-dac4c191ba9b',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_7a3bafc7-ae0a-4c96-82bf-0e16ba8ff97d',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_1c0a359a-f05b-4fa7-8cac-b0018a68ee74',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_920b7d87-7487-47b2-b46a-0727145161ce',
'e1bd9ca9-9d3b-44b4-9c17-ae345f663e27_b9ddf35b-8fa1-4f45-a41d-c873e600ca7c'    
]

unify_tables(url = url, api_token = api_token, datapool = datapool, tables = tables, tableSchema = tableSchema, function = function)

2020-10-15 15:15:01 - pycelonis: Login successful! Hello josef.rieger@cbs-consulting.de
2020-10-15 15:15:02 - pycelonis: Best matches: [(0.44, 'PSG 049 - SAP P2P Extractions Full Load'), (0.44, 'PSG 048 - SAP P2P Extractions Full Load'), (0.44, 'PSG 045 - SAP P2P Extractions Full Load')]
The union statement has been saved in the global data job "Unify Tables" for Table BKPF
The union statement has been saved in the global data job "Unify Tables" for Table BSEG
The union statement has been saved in the global data job "Unify Tables" for Table CDHDR
The union statement has been saved in the global data job "Unify Tables" for Table CDPOS
The union statement has been saved in the global data job "Unify Tables" for Table DD02T
The union statement has been saved in the global data job "Unify Tables" for Table DD03M
The union statement has been saved in the global data job "Unify Tables" for Table DD07T
The union statement has been saved in the global data job "Unify Tables" for Table EBAN
Th